In [1]:
 # import libraries
    
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold,  cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

from collections import Counter
from sklearn.datasets import make_classification

In [2]:
df = pd.read_csv('creditcard.csv')
pd.set_option('display.max_columns', None)

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [5]:
df.Class.describe()

count    284807.000000
mean          0.001727
std           0.041527
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Class, dtype: float64

In [6]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [7]:
#x = df.drop('Class', axis = 1)
#y = df['Class']

# selecting the 19 most important features according to our EDA 
X = df[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', \
        'V16', 'V17', 'V21', 'V23', 'V27']]

y = df['Class'] # selecting the target variable

In [8]:
#split the data into training and testing sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1,stratify = y,)

In [9]:
y_train.value_counts()

0    213236
1       369
Name: Class, dtype: int64

In [10]:
print(y_train.value_counts(), '\n', y_test.value_counts())


0    213236
1       369
Name: Class, dtype: int64 
 0    71079
1      123
Name: Class, dtype: int64


#### Working on the training set before sampling

In [11]:
df2 = pd.concat([X_train,y_train], axis = 1) #carrying out concating before sampling

In [12]:
df2 = df2.reset_index(drop = True)

In [13]:
df2.shape

(213605, 20)

In [14]:
df2.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V16,V17,V21,V23,V27,Class
0,0.880047,-0.21763,0.381760,1.423115,-0.315539,0.060854,0.121998,0.090528,0.014026,-0.008752,1.335037,1.254618,-0.299866,0.315214,-0.568017,0.094930,0.033884,-0.152663,0.013748,0
1,1.199802,0.28808,0.406009,0.627884,-0.440308,-0.872345,-0.033785,-0.051812,-0.193606,-0.104671,1.760868,0.823998,-0.207848,-0.016464,0.630317,-0.164246,-0.226447,0.146230,-0.028580,0


In [15]:
one = df2[df2.iloc[:,19]==1] # index out values of class 1 (one)

In [16]:
one.iloc[:,19].value_counts()

1    369
Name: Class, dtype: int64

In [17]:
zero = df2[df2.iloc[:,19]==0] # index out values of class 0 (zero)

In [18]:
zero.iloc[:,19].value_counts()

0    213236
Name: Class, dtype: int64

### Sampling

In [19]:
zero = zero.iloc[:10000,:] # taking 10000 samples

In [20]:
one.shape

(369, 20)

In [21]:
zero.shape

(10000, 20)

In [22]:
training = pd.concat([one,zero], axis = 0)

In [23]:
training.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V16,V17,V21,V23,V27,Class
17,-13.192671,12.785971,-9.906650,3.320337,-4.801176,5.760059,-18.750889,-37.353443,-0.391540,-5.052502,4.406806,-4.610756,-1.909488,-9.072711,-6.211557,-6.248145,27.202839,5.303607,1.269566,1
965,0.467992,1.100118,-5.607145,2.204714,-0.578539,-0.174200,-3.454201,1.102823,-1.065016,-5.416037,4.497929,-5.019610,-1.019691,-7.914989,-4.472014,-5.856998,0.983481,-0.285103,0.851798,1
1063,-4.320609,3.199939,-5.799736,6.502330,0.378479,-1.948246,-2.167860,-0.728207,-1.977238,-3.473411,4.569194,-9.321153,-1.592518,-14.266836,-4.066209,-6.626968,0.476660,-0.136940,-2.649107,1
1705,1.243848,0.524526,-0.538884,1.209196,0.479538,-0.197429,0.049166,0.037792,0.128119,-0.552903,-0.668359,-0.818804,-0.883832,-0.994581,0.624003,0.705794,-0.051660,-0.192846,0.045917,1
1773,-0.758469,-0.045410,-0.168438,-1.313275,-1.901763,0.739433,3.071892,-0.483422,0.618203,-1.769060,-0.651414,-0.005423,-0.517194,0.217470,-1.173978,0.243347,0.042619,0.072229,0.150606,1


In [24]:
training = training.reset_index(drop = True)

In [25]:
import sklearn.utils
training = sklearn.utils.shuffle(training) # carrying out shuffling
# reseting the index
training = training.reset_index(drop=True)

In [26]:
training.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V16,V17,V21,V23,V27,Class
0,-0.762830,1.501434,-1.277413,-1.057065,0.285690,-0.623525,0.486517,0.702995,-0.048733,-0.784035,-1.530789,0.416282,0.270682,0.734383,-0.111114,-0.159829,-0.119004,0.019868,0.066645,0
1,-4.641893,2.902086,-1.572939,2.507299,-0.871783,-1.040903,-1.593901,-3.254905,1.908963,1.077418,3.338502,-6.542610,1.099536,-3.266476,-4.842383,-5.269876,1.963597,-0.549340,-3.908080,1
2,-3.878026,2.563627,-1.537670,1.118652,-1.964399,-1.119874,-1.163903,2.205631,-0.871067,-0.312159,-1.252042,0.969161,0.674912,1.850492,0.399301,0.726712,0.230255,0.280281,-0.474066,0
3,-3.904733,-3.918511,1.197331,-1.800252,3.028695,0.550643,-0.902378,0.333067,-0.450413,0.248008,0.750522,-0.283080,-0.326333,-0.751312,0.765086,0.269544,-0.599232,1.604914,0.197400,0
4,1.274782,0.239294,0.126050,0.981467,0.308615,0.288049,0.070181,-0.096043,0.315763,-0.220228,-1.673102,0.968872,1.900641,-0.477572,0.038797,-0.674142,-0.321162,-0.175115,0.045943,0


In [27]:
new_Xtrain = training.iloc[:,:19]
new_ytrain = training.iloc[:,19]

In [28]:
new_ytrain.head(2)

0    0
1    1
Name: Class, dtype: int64

In [29]:
new_ytrain.value_counts()

0    10000
1      369
Name: Class, dtype: int64

In [30]:
import imblearn
from imblearn.over_sampling import SMOTE

#smote = SMOTE(random_state=1) # defining an instance

#x_train_balanced, y_balanced = smote.fit_sample(new_Xtrain, new_ytrain)

In [31]:
#x_train_balanced = pd.DataFrame(new_Xtrain, columns=new_Xtrain.columns)
#y_balanced = pd.Series(y_balanced)
#y_balanced.value_counts()

In [32]:
#x_train_balanced.head()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalised_train_df = scaler.fit_transform(new_Xtrain)
normalised_train_df = pd.DataFrame(normalised_train_df, columns=new_Xtrain.columns)


In [33]:
X_test = X_test.reset_index(drop=True)

normalised_test_df = scaler.transform(X_test)
normalised_test_df = pd.DataFrame(normalised_test_df, columns=X_test.columns)

In [34]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix, classification_report


In [53]:
# xgboost with random forest
from xgboost import XGBRFClassifier
extreme = XGBRFClassifier(random_state = 1)
extreme.fit(normalised_train_df, new_ytrain)
extreme_pred = extreme.predict(normalised_test_df)

In [36]:
#classification report
print(classification_report(y_test, extreme_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71079
           1       0.74      0.90      0.81       123

    accuracy                           1.00     71202
   macro avg       0.87      0.95      0.91     71202
weighted avg       1.00      1.00      1.00     71202



In [60]:
#xgboost using gradient boosting
from xgboost import XGBClassifier
extreme1 = XGBClassifier(random_state = 1)
extreme1.fit(normalised_train_df, new_ytrain)
extreme1_pred = extreme1.predict(normalised_test_df)

In [38]:
#classification report
print(classification_report(y_test, extreme1_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71079
           1       0.61      0.89      0.73       123

    accuracy                           1.00     71202
   macro avg       0.81      0.95      0.86     71202
weighted avg       1.00      1.00      1.00     71202



In [42]:
confusion_matrix(y_test, extreme1_pred)

array([[71010,    69],
       [   13,   110]], dtype=int64)

In [56]:
from sklearn.metrics import average_precision_score

average_precision_score(y_test, extreme_pred)

0.6679734126405055

In [57]:
print('f1 score:', f1_score(y_test, extreme_pred))

f1 score: 0.8131868131868132


In [61]:
kfold = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

# use area under the precision-recall curve to show classification accuracy
scoring = 'roc_auc'
results = cross_val_score(extreme1, X, y, cv=kfold, scoring = scoring,n_jobs = 4)
print( "AUC: %.3f (%.3f)" % (results.mean(), results.std()))

AUC: 0.976 (0.007)


### Cost Saving

In [62]:


# computing the class weight

val_count = df['Class'].value_counts()
weights = dict(1 / val_count)
weights



{0: 3.51722561243691e-06, 1: 0.0020325203252032522}

In [63]:
# adminstrative cost
admin_cost = 2.5

In [64]:
# defining a function to calculate cost savings
def cost_saving(ytrue, ypred, amount):
    fp = np.sum((ytrue == 0) & (ypred == 1))
    cost = np.sum(fp*admin_cost) + np.sum((amount[(ytrue == 1) & (ypred == 0)]))
    max_cost = np.sum((amount[(ytrue == 1)]))
    savings = 1 - (cost/max_cost)
    
    return savings

In [65]:
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)

In [66]:
# defining a function to calculate cost saving per fold (splits) of our cv
def cost_saving_per_split(scores, x, y, cv_object):
    results = []
    for i, (_, test_ind) in zip(range(cv_object.n_splits), cv_object.split(x, y)):
        ypred = scores['estimator'][i].predict(x[test_ind])
        ytrue = y[test_ind]
        amount = df['Amount'].values[test_ind]
        results.append(cost_saving(ytrue, ypred, amount))
        
    return results

In [67]:
# defining a function to return a dataframe of metrics results for each fold in our cv
def get_metric_scores(scores, x, y=y, cv_object=cv):
    ind = ['split_'+str(n) for n in range(1, cv_object.n_splits+1)]
    
    scores_credit = pd.DataFrame(index=ind)
    
    scores_credit['f1_score'] = scores['test_f1']
    scores_credit['auc_pr'] = scores['test_average_precision']
    scores_credit['cost_savings'] = cost_saving_per_split(scores, x, y, cv_object)

    return scores_credit

In [68]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier


In [69]:
fraud_sensitive_model = XGBClassifier(random_state = 1,class_weight=weights)
fraud_sensitive_scaler = StandardScaler()

fraud_sensitive_pipe = Pipeline([('scaler', fraud_sensitive_scaler), ('model', fraud_sensitive_model)])
fraud_sensitive_scores = cross_validate(fraud_sensitive_pipe, np.array(X), y, 
                                        scoring=['f1', 'average_precision'], 
                                        cv=cv, n_jobs=4, return_estimator=True,error_score='raise')

In [70]:
fraud_sensitive_results = get_metric_scores(fraud_sensitive_scores, np.array(X))
fraud_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.827273,0.817905,0.669423
split_2,0.849315,0.825010,0.634392
split_3,0.834081,0.841838,0.752930
split_4,0.898305,0.900731,0.662691


In [71]:
fraud_sensitive_results.mean()

f1_score        0.852243
auc_pr          0.846371
cost_savings    0.679859
dtype: float64

In [72]:
sample_weights = np.array([df['Amount'][ind] if fraud else admin_cost for ind, fraud in enumerate(y)])

In [73]:
scaler2 = StandardScaler()
cost_sensitive_model = XGBClassifier(random_state = 1)

cost_sensitive_pipe = Pipeline([('scaler', scaler2), ('model', cost_sensitive_model)])
cost_sensitive_scores = cross_validate(cost_sensitive_model, np.array(X), y, 
                                       scoring=['f1', 'average_precision'], 
                                       cv=cv, n_jobs=4, return_estimator=True, 
                                       fit_params={'sample_weight': sample_weights})

In [74]:
#Using 'log' as loss function
cost_sensitive_results = get_metric_scores(cost_sensitive_scores, np.array(X))
cost_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.842105,0.808312,0.795350
split_2,0.862222,0.827995,0.681225
split_3,0.816143,0.833999,0.757035
split_4,0.902954,0.890174,0.731259


In [75]:
cost_sensitive_results.mean()

f1_score        0.855856
auc_pr          0.840120
cost_savings    0.741217
dtype: float64